In [ ]:
import os
import pickle
import csv
import sys

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import scipy.stats as sps

import chippr

%matplotlib inline

# making CosmoLike input

In [ ]:
test_dir = '../results/all_systematics'

In [ ]:
simulated_posteriors = chippr.catalog(params='all_systematics.txt', loc=test_dir)

In [ ]:
data = simulated_posteriors.read(loc='data', style='.txt')

In [ ]:
params = chippr.utils.ingest('all_systematics.txt')
def check_prob_params(params):
    """
    Sets parameter values pertaining to components of probability

    Parameters
    ----------
    params: dict
        dictionary containing key/value pairs for probability

    Returns
    -------
    params: dict
        dictionary containing key/value pairs for probability
    """
    if 'prior_mean' not in params:
        params['prior_mean'] = 'interim'
    else:
        params['prior_mean'] = params['prior_mean'][0]
    if 'no_prior' not in params:
        params['no_prior'] = 0
    else:
        params['no_prior'] = int(params['no_prior'][0])
    if 'no_data' not in params:
        params['no_data'] = 0
    else:
        params['no_data'] = int(params['no_data'][0])
    return params
params = check_prob_params(params)
def set_up_prior(data, params):
    """
    Function to create prior distribution from data

    Parameters
    ----------
    data: dict
        catalog dictionary containing bin endpoints, log interim prior, and log
        interim posteriors
    params: dict
        dictionary of parameter values for creation of prior

    Returns
    -------
    prior: chippr.mvn object
        prior distribution as multivariate normal
    """
    zs = data['bin_ends']
    log_nz_intp = data['log_interim_prior']
    log_z_posts = data['log_interim_posteriors']

    z_difs = zs[1:]-zs[:-1]
    z_mids = (zs[1:]+zs[:-1])/2.
    n_bins = len(z_mids)

    n_pdfs = len(log_z_posts)

    a = 1.# / n_bins
    b = 20.#1. / z_difs ** 2
    c = a / n_pdfs
    prior_var = np.eye(n_bins)
    for k in range(n_bins):
        prior_var[k] = a * np.exp(-0.5 * b * (z_mids[k] - z_mids) ** 2)
    prior_var += c * np.identity(n_bins)

    prior_mean = log_nz_intp
    prior = chippr.mvn(prior_mean, prior_var)
    if params['prior_mean'] == 'sample':
        new_mean = prior.sample_one()
        prior = chippr.mvn(new_mean, prior_var)
        print(params['prior_mean'], prior_mean, new_mean)
    else:
        print(params['prior_mean'], prior_mean)

    return (prior, prior_var)
(prior, cov) = set_up_prior(data, params)

In [ ]:
with open(os.path.join('../results/all_systematics/data', 'true_params.p'), 'r') as true_file:
    true_nz_info = pickle.load(true_file)

In [ ]:
true_funcs = [chippr.discrete(true_nz_info['grid'], true_nz_info['amps'])]
true_amps = [1.]
true_nz = chippr.gmix(true_amps, true_funcs, limits=(min(true_nz_info['bins']), max(true_nz_info['bins'])))
grid_mids = (true_nz_info['grid'][1:] + true_nz_info['grid'][:-1])/2.
plt.plot(grid_mids, true_nz.evaluate(grid_mids))


In [ ]:
nz = chippr.log_z_dens(data, prior, truth=true_nz, loc='../results/all_systematics', vb=True)

In [ ]:
nz.read('nz.p')

In [ ]:
bins_to_write = np.linspace(0.0101, 3.5001, 350)
empty_bins = np.random.random((350, 4))/350.


In [ ]:
bin_mids = (nz.info['bin_ends'][1:] + nz.info['bin_ends'][:-1])/2.
with open(os.path.join('../results/all_systematics/results', 'nz_mmle_test.txt'), 'wb') as cosmolike_file:
#     cosmolike_file.write(zip(bin_mids, np.exp(nz.info['estimators']['log_mmle_nz']))
    cosmolike_writer = csv.writer(cosmolike_file, delimiter=' ')
    cosmolike_writer.writerows(zip(bin_mids, np.exp(nz.info['estimators']['log_mmle_nz'])))

# what to do with CosmoLike output

In [ ]:
covariances = np.zeros((200, 200))+sys.float_info.epsilon
with open(os.path.join('../results/all_systematics/results', 'Cl_cov.nz_histo.txt'), 'rb') as cosmolike_file:
    cosmolike_reader = csv.reader(cosmolike_file, delimiter=' ')
    cosmolike_reader.next()
    for row in cosmolike_reader:
        covariances[int(row[0])][int(row[1])] = float(row[2])

In [ ]:
plt.imshow(np.linalg.pinv(covariances))
# print(covariances)

In [ ]:
# Cl, dCldOm, dClds8, dCldns, dCldw0, dCldwa, dCldOb, dCldH0 = np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200), np.zeros(200)
deriv_info = np.zeros((200, 7))

with open(os.path.join('../results/all_systematics/results', 'Cl_derivs.nz_histo.txt'), 'rb') as cosmolike_file:
    cosmolike_reader = csv.reader(cosmolike_file, delimiter=' ')
    cosmolike_reader.next()
    i = 0
    while i < 200:
        for row in cosmolike_reader:
            for j in range(7):
                deriv_info[i][j] = float(row[j])
        i += 1

In [ ]:
deriv_info = deriv_info.T
dCldOm, dClds8, dCldns, dCldw0, dCldwa, dCldOb, dCldH0 = deriv_info[0], deriv_info[1], deriv_info[2], deriv_info[3], deriv_info[4], deriv_info[5], deriv_info[6]

In [ ]:
ls = np.arange(200)
fisher = np.eye(7)
for i in range(7):
    for j in range(i+1):
        fisher[i][j] = np.sum((2. * ls + 1.) * deriv_info[i] * np.linalg.inv(covariances) * deriv_info[j])
        fisher[j][i] = fisher[i][j]

In [ ]:
plt.imshow(np.linalg.pinv(fisher))

In [ ]:
print(np.min(np.linalg.pinv(fisher)))

In [ ]:
keys = ['dCldOm', 'dClds8', 'dCldns', 'dCldw0', 'dCldwa', 'dCldOb', 'dCldH0']
def mycorner(fisher, keys):
    ncol = len(keys)
    fig = plt.figure(figsize=(ncol*5, ncol*5))
    ax = [[fig.add_subplot(ncol, ncol, ncol * i + j + 1) for j in range(i+1)] for i in range(ncol)]
    for i in range(ncol):
        for j in range(i+1):
            sxx = fisher[i][i]
            sx = np.sqrt(sxx)
            xlim = (-5.*sx, 5.*sx)
            if i == j:
                x_grid = np.linspace(xlim[0], xlim[-1], 100)
                func = sps.norm(0., np.sqrt(sxx))
                ax[i][j].plot(x_grid, func.pdf(x_grid))
                ax[i][j].set_xlabel(keys[i])
            else:
                syy = fisher[j][j]
                sy = np.sqrt(syy)
                ylim = (-5.*sy, 5.*sy)
                term1 = (sxx + syy)/2.
                term2 = np.sqrt((sxx-syy)**2/4. + fisher[i][j] * fisher[j][i])
                a = np.sqrt(np.abs(term1 + term2))
                b = np.sqrt(np.abs(term1 - term2))
                t = np.arctan((fisher[i][j] + fisher[j][i])/(sxx-syy))/2.
                ellipse = Ellipse(xy=(0., 0.), width=2.*a, height=2.*b, angle=t*180./np.pi)
                ax[i][j].add_artist(ellipse)
                ax[i][j].set_xlabel(keys[i])
                ax[i][j].set_ylabel(keys[j])
                ax[i][j].set_xlim(xlim)
                ax[i][j].set_ylim(ylim)
    return
mycorner(np.linalg.pinv(fisher), keys)

# ancient scratch

In [ ]:
z_0 = 0.3
def smooth_func(z):
    return 1/(2 * z_0) * (z/z_0)**2 * np.exp(-z/z_0)
zs = np.linspace(0., 1., 100)

nz = smooth_func(zs[:-1])
nz /= np.dot(nz, zs[1:]-zs[:-1])
plt.plot(zs[:-1], nz)

In [ ]:
print(np.dot(true_funcs[0].evaluate(grid_mids), true_nz_info['grid'][1:]-true_nz_info['grid'][:-1]))

In [ ]:
print(np.dot(true_funcs[0].distweights, true_funcs[0].dbins))

In [ ]:
func = sps.norm(0.25, 0.05)
print(func.std())
x = np.linspace(0., 1., 100)
plt.plot(x, func.pdf(x))

In [ ]:
n_bins = 100
z_mids = np.linspace(0., 1., n_bins)

a = 1.
b = 20.#mid-scale variations, larger means more peaks
c = 1.e-6#longest-scale variation, lower increases amplitude relative to small-scale
prior_var = np.eye(n_bins)
for k in range(n_bins):
    prior_var[k] = a * np.exp(-0.5 * b * (z_mids[k] - z_mids) ** 2)
prior_var += c * np.identity(n_bins)

prior_mean = np.zeros(n_bins)
prior = chippr.mvn(prior_mean, prior_var)

In [ ]:
samples = prior.sample(7)
for each in samples:
    plt.plot(z_mids, each)